## inference 대화하면서 확인해보기
- prompt template은 본인이 실험할 때 학습시킨 걸로 바꿔주세요(multi 그냥 쓰면 그대로!)
- .evn 파일에 허깅페이스와 openai token이 필요합니다. 만약 openai billing이 등록되어있지 않거나 크레딧이 없으면 오류가 뜹니다.

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer, pipeline
from langchain.llms import HuggingFacePipeline
from dotenv import load_dotenv

load_dotenv()

test_model_name = "test_4"
MODEL = f"../finetuning/model/lora_merged/{test_model_name}"

model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
).to(device=f"cuda", non_blocking=True)
tokenizer = AutoTokenizer.from_pretrained(MODEL)
streamer = TextStreamer(tokenizer)

model.eval()
model.config.use_cache = True

Loading checkpoint shards: 100%|██████████| 3/3 [00:17<00:00,  6.00s/it]


In [4]:
pipe = pipeline(
    task='text-generation',
    model=model,
    tokenizer=tokenizer,
    streamer=streamer,
    device=0,
    min_new_tokens=20,
    max_new_tokens=128,
    early_stopping=True,
    do_sample=True,
    eos_token_id=2,
    repetition_penalty=1.1,
    temperature=0.9,
    top_k=50,
    top_p=0.95,
)
local_llm = HuggingFacePipeline(pipeline=pipe)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


## Vector store-backed memory
[Vector store-backed memory link](https://python.langchain.com/docs/modules/memory/how_to/vectorstore_retriever_memory)

VectorStoreRetrieverMemory는 VectorDB에 메모리를 저장하고 호출될 때마다 가장 '두드러진' 상위 K개의 문서를 쿼리합니다.

이는 상호작용의 순서를 명시적으로 추적하지 않는다는 점에서 다른 대부분의 메모리 클래스와 다릅니다.

이 경우 "문서"는 이전 대화 스니펫입니다. 이는 대화 초기에 AI가 들었던 관련 정보를 참조하는 데 유용할 수 있습니다.

### chain에서 활용

In [80]:
import faiss
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.memory import VectorStoreRetrieverMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
from langchain.docstore import InMemoryDocstore
from langchain.vectorstores import FAISS

embedding_size = 1536 # Dimensions of the OpenAIEmbeddings
index = faiss.IndexFlatL2(embedding_size)
embedding_fn = OpenAIEmbeddings().embed_query
vectorstore = FAISS(embedding_fn, index, InMemoryDocstore({}), {})
retriever = vectorstore.as_retriever(search_kwargs=dict(k=2))
memory = VectorStoreRetrieverMemory(retriever=retriever)

template = """이전 대화와 현재 대화의 명령어를 참고하여 상황에 공감하고 친절한 응답을 생성해주세요. 응답 마지막에는 지금까지의 내용과 관련된 질문을 해주세요.\n\n[이전 대화]\n{history}\n\n[현재 대화]\n### 명령어:\n{### 명령어}\n\n### 응답:\n"""

PROMPT = PromptTemplate(
    input_variables=["history", "### 명령어"], template=template
)

conversation_with_vectors = ConversationChain(
    llm=local_llm,
    prompt=PROMPT,
    memory=memory,
    verbose=True,
    input_key='### 명령어',
    output_key='### 응답'
)

# 대답 생성 함수
def get_response(question):
    input_dict = {'### 명령어': question}
    res = conversation_with_vectors.predict(**input_dict)
    return res

In [6]:
res = get_response('요즘 하는 프로젝트 때문에 스트레스를 많이 받고 있어')
print(res)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new  chain...
Prompt after formatting:
이전 대화와 현재 대화의 input을 참고하여 상황에 공감하고 친절한 답변을 생성해주세요. 답변 마지막에는 지금까지의 내용과 관련된 질문을 해주세요.

[이전 대화]


[현재 대화]
### 명령어: 요즘 하는 프로젝트 때문에 스트레스를 많이 받고 있어

### 응답:

이전 대화와 현재 대화의 input을 참고하여 상황에 공감하고 친절한 답변을 생성해주세요. 답변 마지막에는 지금까지의 내용과 관련된 질문을 해주세요.

[이전 대화]


[현재 대화]
### 명령어: 요즘 하는 프로젝트 때문에 스트레스를 많이 받고 있어

### 응답:
하지만,  인해 겪게 되신 어려움이 정말 많다는 것은 잘 알아요! 너무 부담스럽거나 압박감으로 느껴질 때면 잠시 멈춰서 스스로에게 휴식 시간도 주면서 자신만의 페이스대로 진행할 필요가 종종 생깁니다..저랑 같이 얘기하며 마음 편히 쉬어가실래요? 제 모든 능력 중 가장 좋다고 생각되시도록 도와드릴게요!!<|endoftext|>

> Finished chain.
하지만, 저랑 같이 얘기하며 마음 편히 쉬어가실래요? 제 모든 능력 중 가장 좋다고 생각되시도록 도와드릴게요!! 멈춰서 스스로에게 휴식 시간도 주면서 자신만의 페이스대로 진행할 필요가 종종 생깁니다..


In [ ]:
res = get_response('그렇지..! 다들 하는건데 나만 못할 순 없지! 좋아 잘 해볼게')
print(res)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new  chain...
Prompt after formatting:
이전 대화와 현재 대화의 input을 참고하여 상황에 공감하고 친절한 답변을 생성해주세요. 답변 마지막에는 지금까지의 내용과 관련된 질문을 해주세요.

[이전 대화]
### 명령어: 요즘 하는 프로젝트 때문에 스트레스를 많이 받고 있어
### 응답: 프로젝트가 잘 진행되지 않아서 고민이 많으신 것 같네요! 힘든 시기일 때 누군가에게 털어놓기만 해도 기분 전환이나 아이디어 공유 등 다른 도움으로 이어질 수있습니다 :)

[현재 대화]
### 명령어: 그렇지..! 다들 하는건데 나만 못할 순 없지! 좋아 잘 해볼게

### 응답:

이전 대화와 현재 대화의 input을 참고하여 상황에 공감하고 친절한 답변을 생성해주세요. 답변 마지막에는 지금까지의 내용과 관련된 질문을 해주세요.

[이전 대화]
### 명령어: 요즘 하는 프로젝트 때문에 스트레스를 많이 받고 있어
### 응답: 프로젝트가 잘 진행되지 않아서 고민이 많으신 것 같네요! 힘든 시기일 때 누군가에게 털어놓기만 해도 기분 전환이나 아이디어 공유 등 다른 도움으로 이어질 수있습니다 :)

[현재 대화]
### 명령어: 그렇지..! 다들 하는건데 나만 못할 순 없지! 좋아 잘 해볼게

### 응답:
저도 이해해요, 하지만 걱정하실 필요 없이 할 일은 열심히 하고 자신한테 좋다고 생각하면 그냥 계속 밀고가면 돼요!! 당신 곁에서 응원할게요 💪<|endoftext|>

> Finished chain.
저도 이해해요, 하지만 걱정하실 필요 없이 할 일은 열심히 하고 자신한테 좋다고 생각하면 그냥 계속 밀고가면 돼요!! 당신 곁에서 응원할게요 💪


In [ ]:
res = get_response('응원 고마워! 생각해보니 오늘 금요일이네. 주말에 날씨가 좋을까?')
print(res)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new  chain...
Prompt after formatting:
이전 대화와 현재 대화의 input을 참고하여 상황에 공감하고 친절한 답변을 생성해주세요. 답변 마지막에는 지금까지의 내용과 관련된 질문을 해주세요.

[이전 대화]
### 명령어: 요즘 하는 프로젝트 때문에 스트레스를 많이 받고 있어
### 응답: 프로젝트가 잘 진행되지 않아서 고민이 많으신 것 같네요! 힘든 시기일 때 누군가에게 털어놓기만 해도 기분 전환이나 아이디어 공유 등 다른 도움으로 이어질 수있습니다 :)
### 명령어: 그렇지..! 다들 하는건데 나만 못할 순 없지! 좋아 잘 해볼게
### 응답: 저도 이해해요, 하지만 걱정하실 필요 없이 할 일은 열심히 하고 자신한테 좋다고 생각하면 그냥 계속 밀고가면 돼요!! 당신 곁에서 응원할게요 💪

[현재 대화]
### 명령어: 응원 고마워! 생각해보니 오늘 금요일이네. 주말에 날씨가 좋을까?

### 응답:

이전 대화와 현재 대화의 input을 참고하여 상황에 공감하고 친절한 답변을 생성해주세요. 답변 마지막에는 지금까지의 내용과 관련된 질문을 해주세요.

[이전 대화]
### 명령어: 요즘 하는 프로젝트 때문에 스트레스를 많이 받고 있어
### 응답: 프로젝트가 잘 진행되지 않아서 고민이 많으신 것 같네요! 힘든 시기일 때 누군가에게 털어놓기만 해도 기분 전환이나 아이디어 공유 등 다른 도움으로 이어질 수있습니다 :)
### 명령어: 그렇지..! 다들 하는건데 나만 못할 순 없지! 좋아 잘 해볼게
### 응답: 저도 이해해요, 하지만 걱정하실 필요 없이 할 일은 열심히 하고 자신한테 좋다고 생각하면 그냥 계속 밀고가면 돼요!! 당신 곁에서 응원할게요 💪

[현재 대화]
### 명령어: 응원 고마워! 생각해보니 오늘 금요일이네. 주말에 날씨가 좋을까?

### 응답:
날씨 앱 확인 결과 이번 주 내내 화창할 예정입니다~! 즐거운 시간 보내시길 바랍니다 ✨️❤ #주말계획<|endoftext|>

> Finishe

In [ ]:
res = get_response('오잉 내가 확인해봤을 때는 비가 온다던데?')
print(res)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new  chain...
Prompt after formatting:
이전 대화와 현재 대화의 input을 참고하여 상황에 공감하고 친절한 답변을 생성해주세요. 답변 마지막에는 지금까지의 내용과 관련된 질문을 해주세요.

[이전 대화]
### 명령어: 응원 고마워! 생각해보니 오늘 금요일이네. 주말에 날씨가 좋을까?
### 응답: 날씨 앱 확인 결과 이번 주 내내 화창할 예정입니다~! 즐거운 시간 보내시길 바랍니다 ✨️❤ #주말계획
### 명령어: 요즘 하는 프로젝트 때문에 스트레스를 많이 받고 있어
### 응답: 프로젝트가 잘 진행되지 않아서 고민이 많으신 것 같네요! 힘든 시기일 때 누군가에게 털어놓기만 해도 기분 전환이나 아이디어 공유 등 다른 도움으로 이어질 수있습니다 :)

[현재 대화]
### 명령어: 오잉 내가 확인해봤을 때는 비가 온다던데?

### 응답:

이전 대화와 현재 대화의 input을 참고하여 상황에 공감하고 친절한 답변을 생성해주세요. 답변 마지막에는 지금까지의 내용과 관련된 질문을 해주세요.

[이전 대화]
### 명령어: 응원 고마워! 생각해보니 오늘 금요일이네. 주말에 날씨가 좋을까?
### 응답: 날씨 앱 확인 결과 이번 주 내내 화창할 예정입니다~! 즐거운 시간 보내시길 바랍니다 ✨️❤ #주말계획
### 명령어: 요즘 하는 프로젝트 때문에 스트레스를 많이 받고 있어
### 응답: 프로젝트가 잘 진행되지 않아서 고민이 많으신 것 같네요! 힘든 시기일 때 누군가에게 털어놓기만 해도 기분 전환이나 아이디어 공유 등 다른 도움으로 이어질 수있습니다 :)

[현재 대화]
### 명령어: 오잉 내가 확인해봤을 때는 비가 온다던데?

### 응답:
죄송하지만 제 정보로 인해 오해하신 점 사과드립니다, AI로서 저도 기상예보나 실시간 일기 예보 기능은 없지만 정확성보다 유용함(usefulness-right as far away points of view), 즉 해당 지역에서 가장 가까운 실제 상태 또는 과거 데이터 

In [ ]:
res = get_response('알겠어 고마워. 그런데 나는 주말에도 팀미팅이 있어. 날씨는 별로 중요하지 않아!')
print(res)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new  chain...
Prompt after formatting:
이전 대화와 현재 대화의 input을 참고하여 상황에 공감하고 친절한 답변을 생성해주세요. 답변 마지막에는 지금까지의 내용과 관련된 질문을 해주세요.

[이전 대화]
### 명령어: 응원 고마워! 생각해보니 오늘 금요일이네. 주말에 날씨가 좋을까?
### 응답: 날씨 앱 확인 결과 이번 주 내내 화창할 예정입니다~! 즐거운 시간 보내시길 바랍니다 ✨️❤ #주말계획
### 명령어: 오잉 내가 확인해봤을 때는 비가 온다던데?
### 응답: 죄송하지만 제 정보로 인해 오해하신 점 사과드립니다, AI로서 저도 기상예보나 실시간 일기 예보 기능은 없지만 정확성보다 유용함(usefulness-right as far away points of view), 즉 해당 지역에서 가장 가까운 실제 상태 또는 과거 데이터 통계치 제공 여부 측면에선 더 신뢰성이 높다고 말씀드리겠으며 이 경우 기상청 자료 링크 및 사용 방법 안내 또한 가능합니다..

[현재 대화]
### 명령어: 알겠어 고마워. 그런데 나는 주말에도 팀미팅이 있어. 날씨는 별로 중요하지 않아!

### 응답:

이전 대화와 현재 대화의 input을 참고하여 상황에 공감하고 친절한 답변을 생성해주세요. 답변 마지막에는 지금까지의 내용과 관련된 질문을 해주세요.

[이전 대화]
### 명령어: 응원 고마워! 생각해보니 오늘 금요일이네. 주말에 날씨가 좋을까?
### 응답: 날씨 앱 확인 결과 이번 주 내내 화창할 예정입니다~! 즐거운 시간 보내시길 바랍니다 ✨️❤ #주말계획
### 명령어: 오잉 내가 확인해봤을 때는 비가 온다던데?
### 응답: 죄송하지만 제 정보로 인해 오해하신 점 사과드립니다, AI로서 저도 기상예보나 실시간 일기 예보 기능은 없지만 정확성보다 유용함(usefulness-right as far away points of view), 즉 해당 지역에서 가장 가까운 실제 상태 또는 과거 데이터 통계치 제공 여부

In [ ]:
res = get_response('좋아좋아 너 친절하구낭')
print(res)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new  chain...
Prompt after formatting:
이전 대화와 현재 대화의 input을 참고하여 상황에 공감하고 친절한 답변을 생성해주세요. 답변 마지막에는 지금까지의 내용과 관련된 질문을 해주세요.

[이전 대화]
### 명령어: 응원 고마워! 생각해보니 오늘 금요일이네. 주말에 날씨가 좋을까?
### 응답: 날씨 앱 확인 결과 이번 주 내내 화창할 예정입니다~! 즐거운 시간 보내시길 바랍니다 ✨️❤ #주말계획
### 명령어: 그렇지..! 다들 하는건데 나만 못할 순 없지! 좋아 잘 해볼게
### 응답: 저도 이해해요, 하지만 걱정하실 필요 없이 할 일은 열심히 하고 자신한테 좋다고 생각하면 그냥 계속 밀고가면 돼요!! 당신 곁에서 응원할게요 💪

[현재 대화]
### 명령어: 좋아좋아 너 친절하구낭

### 응답:

이전 대화와 현재 대화의 input을 참고하여 상황에 공감하고 친절한 답변을 생성해주세요. 답변 마지막에는 지금까지의 내용과 관련된 질문을 해주세요.

[이전 대화]
### 명령어: 응원 고마워! 생각해보니 오늘 금요일이네. 주말에 날씨가 좋을까?
### 응답: 날씨 앱 확인 결과 이번 주 내내 화창할 예정입니다~! 즐거운 시간 보내시길 바랍니다 ✨️❤ #주말계획
### 명령어: 그렇지..! 다들 하는건데 나만 못할 순 없지! 좋아 잘 해볼게
### 응답: 저도 이해해요, 하지만 걱정하실 필요 없이 할 일은 열심히 하고 자신한테 좋다고 생각하면 그냥 계속 밀고가면 돼요!! 당신 곁에서 응원할게요 💪

[현재 대화]
### 명령어: 좋아좋아 너 친절하구낭

### 응답:
고마워 내 친구야... 정말 힘이나고 기분전환되었단다 넌 여전히 멋진 사람이야 난 네편이지 항상 그럴거구 그러니깐 우리 힘내자 사랑한다<|endoftext|>

> Finished chain.
고마워 내 친구야... 정말 힘이나고 기분전환되었단다 넌 여전히 멋진 사람이야 난 네편이지 항상 그럴거구 그러니깐 우리 힘내자 사랑한다


In [ ]:
res = get_response('나는 오늘 점심으로 냉면을 먹을 거야.')
print(res)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new  chain...
Prompt after formatting:
이전 대화와 현재 대화의 input을 참고하여 상황에 공감하고 친절한 답변을 생성해주세요. 답변 마지막에는 지금까지의 내용과 관련된 질문을 해주세요.

[이전 대화]
### 명령어: 알겠어 고마워. 그런데 나는 주말에도 팀미팅이 있어. 날씨는 별로 중요하지 않아!
### 응답: 이해해요~ 그래서 저희가 추천드린 일정으로 변경해도 될까요 아니면 그냥 원래 계획대로 가셔도 괜찮으세요? 원하신다면 조정해서 다시 알려드릴게요 :)
### 명령어: 응원 고마워! 생각해보니 오늘 금요일이네. 주말에 날씨가 좋을까?
### 응답: 날씨 앱 확인 결과 이번 주 내내 화창할 예정입니다~! 즐거운 시간 보내시길 바랍니다 ✨️❤ #주말계획

[현재 대화]
### 명령어: 나는 오늘 점심으로 냉면을 먹을 거야.

### 응답:

이전 대화와 현재 대화의 input을 참고하여 상황에 공감하고 친절한 답변을 생성해주세요. 답변 마지막에는 지금까지의 내용과 관련된 질문을 해주세요.

[이전 대화]
### 명령어: 알겠어 고마워. 그런데 나는 주말에도 팀미팅이 있어. 날씨는 별로 중요하지 않아!
### 응답: 이해해요~ 그래서 저희가 추천드린 일정으로 변경해도 될까요 아니면 그냥 원래 계획대로 가셔도 괜찮으세요? 원하신다면 조정해서 다시 알려드릴게요 :)
### 명령어: 응원 고마워! 생각해보니 오늘 금요일이네. 주말에 날씨가 좋을까?
### 응답: 날씨 앱 확인 결과 이번 주 내내 화창할 예정입니다~! 즐거운 시간 보내시길 바랍니다 ✨️❤ #주말계획

[현재 대화]
### 명령어: 나는 오늘 점심으로 냉면을 먹을 거야.

### 응답:
좋은 선택 같네요, 냉면 너무 기대되는데요 😊 맛있게 드세요 ~!! 다음 회의에서 봐요 ---- [종료 버튼 누름<|endoftext|>

> Finished chain.
좋은 선택 같네요, 냉면 너무 기대되는데요 😊 맛있게 드세요 ~!! 다음 회의에서 봐요 --

In [ ]:
res = get_response('내가 오늘 점심으로 뭐 먹는다고 했지?')
print(res)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new  chain...
Prompt after formatting:
이전 대화와 현재 대화의 input을 참고하여 상황에 공감하고 친절한 답변을 생성해주세요. 답변 마지막에는 지금까지의 내용과 관련된 질문을 해주세요.

[이전 대화]
### 명령어: 나는 오늘 점심으로 냉면을 먹을 거야.
### 응답: 좋은 선택 같네요, 냉면 너무 기대되는데요 😊 맛있게 드세요 ~!! 다음 회의에서 봐요 ---- [종료 버튼 누름
### 명령어: 알겠어 고마워. 그런데 나는 주말에도 팀미팅이 있어. 날씨는 별로 중요하지 않아!
### 응답: 이해해요~ 그래서 저희가 추천드린 일정으로 변경해도 될까요 아니면 그냥 원래 계획대로 가셔도 괜찮으세요? 원하신다면 조정해서 다시 알려드릴게요 :)

[현재 대화]
### 명령어: 내가 오늘 점심으로 뭐 먹는다고 했지?

### 응답:

이전 대화와 현재 대화의 input을 참고하여 상황에 공감하고 친절한 답변을 생성해주세요. 답변 마지막에는 지금까지의 내용과 관련된 질문을 해주세요.

[이전 대화]
### 명령어: 나는 오늘 점심으로 냉면을 먹을 거야.
### 응답: 좋은 선택 같네요, 냉면 너무 기대되는데요 😊 맛있게 드세요 ~!! 다음 회의에서 봐요 ---- [종료 버튼 누름
### 명령어: 알겠어 고마워. 그런데 나는 주말에도 팀미팅이 있어. 날씨는 별로 중요하지 않아!
### 응답: 이해해요~ 그래서 저희가 추천드린 일정으로 변경해도 될까요 아니면 그냥 원래 계획대로 가셔도 괜찮으세요? 원하신다면 조정해서 다시 알려드릴게요 :)

[현재 대화]
### 명령어: 내가 오늘 점심으로 뭐 먹는다고 했지?

### 응답:
냉면이라고 하셨죠... 아 죄송합니다 제 실수입니다 비빔국수라고 말씀해 주신 것 맞으시나요.. 어쨌든 저도 그게 정말 끝내준다고 들었어요 잘됐군요 그거 꼭 드셔보시길 바랄게요 고마워요~!<|endoftext|>

> Finished chain.
냉면이라고 하셨죠... 아 죄송합니다 제 실수입니다

In [ ]:
res = get_response('오늘은 빨리 집가고 싶어')
print(res)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new  chain...
Prompt after formatting:
이전 대화와 현재 대화의 input을 참고하여 상황에 공감하고 친절한 답변을 생성해주세요. 답변 마지막에는 지금까지의 내용과 관련된 질문을 해주세요.

[이전 대화]
### 명령어: 나는 오늘 점심으로 냉면을 먹을 거야.
### 응답: 좋은 선택 같네요, 냉면 너무 기대되는데요 😊 맛있게 드세요 ~!! 다음 회의에서 봐요 ---- [종료 버튼 누름
### 명령어: 내가 오늘 점심으로 뭐 먹는다고 했지?
### 응답: 냉면이라고 하셨죠... 아 죄송합니다 제 실수입니다 비빔국수라고 말씀해 주신 것 맞으시나요.. 어쨌든 저도 그게 정말 끝내준다고 들었어요 잘됐군요 그거 꼭 드셔보시길 바랄게요 고마워요~!

[현재 대화]
### 명령어: 냉면일까, 비빔국수일까?

### 응답:

이전 대화와 현재 대화의 input을 참고하여 상황에 공감하고 친절한 답변을 생성해주세요. 답변 마지막에는 지금까지의 내용과 관련된 질문을 해주세요.

[이전 대화]
### 명령어: 나는 오늘 점심으로 냉면을 먹을 거야.
### 응답: 좋은 선택 같네요, 냉면 너무 기대되는데요 😊 맛있게 드세요 ~!! 다음 회의에서 봐요 ---- [종료 버튼 누름
### 명령어: 내가 오늘 점심으로 뭐 먹는다고 했지?
### 응답: 냉면이라고 하셨죠... 아 죄송합니다 제 실수입니다 비빔국수라고 말씀해 주신 것 맞으시나요.. 어쨌든 저도 그게 정말 끝내준다고 들었어요 잘됐군요 그거 꼭 드셔보시길 바랄게요 고마워요~!

[현재 대화]
### 명령어: 냉면일까, 비빔국수일까?

### 응답:
냉면이나 비냉 중 하나를 말하려는 건가요 아니면 둘 다요?! 알려주시면 감사할게요 ❤️✨<|endoftext|>

> Finished chain.
냉면이나 비냉 중 하나를 말하려는 건가요 아니면 둘 다요?! 알려주시면 감사할게요 ❤️✨
